[{'title': 'Maths Past Papers', 'start': '09:00', 'end': '09:45', 'duration': 45, 'focus': 'Past papers practice'}, {'title': 'Chapter Reading', 'start': '10:00', 'end': '10:30', 'duration': 30, 'focus': 'Read weak chapters'}]


In [3]:



import gradio as gr

from backend import (
    generate_study_plan,
    generate_quote,
    save_study_plan,
    generate_quick_insights,
    load_study_plan,
    load_habit_data,
    log_study_session,
    calculate_total_hours,
    calculate_streak,
    generate_weekly_summary,
    complete_current_task
)




def load_dashboard():
        logs = load_habit_data()

        # ---- KPIs ----
        total_hours = calculate_total_hours(logs) if logs else 0
        streak = calculate_streak(logs) if logs else 0

        
        quote = generate_quote()
        quote_html = f"<div class='quote-bubble'>{quote}</div>"
        hours_html = f"<div class='kpi-box'>Hours Studied<br><b>{total_hours}</b></div>"
        streak_html = f"<div class='kpi-box'>Study Streak<br><b>{streak} days</b></div>"

        plan_data = load_study_plan()

        if plan_data:
            next_task = plan_data.get("next_task")
            task_html = (
                f"<div class='next-task'>"
                f" Your next task:<br><b>{next_task}</b>"
                f"</div>"
            )
        else:
            task_html = (
                "<div class='next-task'>"
                "Generate a study plan to see your next task"
                "</div>"
            )


        return quote_html, hours_html, streak_html, task_html
# -------------------------
# App Layout
# -------------------------
def load_css(theme):
    filename = "assets/dark.css" if theme == "Dark" else "assets/light.css"
    with open(filename, "r") as f:
        return f.read()

dark_css = load_css("Dark")
light_css = load_css("Light")
def switch_theme(choice):
    css = light_css if choice == "Light" else dark_css
    return f"<style>{css}</style>"


with gr.Blocks() as demo:
    css_box = gr.HTML(f"<style>{dark_css}</style>")

    theme_state = gr.State("Dark")
    with gr.Tabs():
        
 
        # -----------------------------------------
        # HOME TAB
        # -----------------------------------------
        with gr.Tab("Home"):

            # Header
            gr.Markdown("<div class='sense-header'> SenseFlow Dashboard</div>")

            # Quote bubble
            quote_box = gr.Markdown("<div class='quote-bubble'> Quote loading...</div>")
            
           
            theme_toggle = gr.Radio(
            ["Dark", "Light"],
            value="Dark",
            label="Theme",
            elem_classes="theme-toggle"
        )
            
          


            # KPI + Donut layout
            with gr.Row():

                # LEFT SIDE KPIs
                with gr.Column(scale=1):
                    kpi_hours = gr.Markdown("<div class='kpi-box'>Hours Studied: --</div>")
                    kpi_streak = gr.Markdown("<div class='kpi-box'>Study Streak: --</div>")

                # RIGHT SIDE DONUT
                with gr.Column(scale=2):
                    gr.Markdown(
                        "<div style='color:white; font-size:20px; margin-bottom:10px;'>Progress Donut</div>"
                    )
                    donut_placeholder = gr.Markdown(
                        "<div style='color:white;'> Donut chart will appear here (Tasks Completed vs Tasks Left)</div>"
                    )

            # NEXT TASK BOX
            next_task_box = gr.Markdown(
                "<div class='next-task'>📌 Your next task is...<br>(Will be populated from study plan)</div>"
            )

            # NAVIGATION BUTTONS
            gr.Markdown("<div style='color:white; font-size:20px; margin-top:25px;'>Navigation</div>")

            with gr.Row():
                planner_btn = gr.Button("📘 Generate Study Plan", elem_classes="nav-btn")
                calendar_btn = gr.Button("📅 Calendar ", elem_classes="nav-btn")
                tracker_btn = gr.Button("📝 Log Studies", elem_classes="nav-btn")
                complete_task_btn = gr.Button("✅ Complete Task")
                complete_task_btn.click(
                    fn=complete_current_task,
                    inputs=None,
                    outputs=next_task_box
                )


                refresh_dashboard_btn = gr.Button("🔄 Refresh Dashboard", elem_classes="nav-btn")
                refresh_dashboard_btn.click(
                fn=load_dashboard,
                inputs=None,
                outputs=[quote_box, kpi_hours, kpi_streak, next_task_box]
            )

                quote_btn = gr.Button("🔄 Refresh Quote", elem_classes="nav-btn")
                quote_btn.click(
                fn=lambda: f"<div class='quote-bubble'>{generate_quote()}</div>",
                inputs=None,
                outputs=quote_box
)
       
        
        # -----------------------------------------
        # OTHER TABS (unchanged)
        # -----------------------------------------
        with gr.Tab("Study Planner"):

            gr.Markdown("<div class='planner-title'>STUDY PLANNER</div>")

            # ---- Add Task UI ----
            new_task = gr.Textbox(label="Add a Task")
            add_task_btn = gr.Button("➕ Add Task", elem_classes="nav-btn")

            # This holds the tasks internally
            task_list_state = gr.State([])

            # Where tasks will be shown on screen
            task_list_display = gr.Markdown("<div class='task-list'>No tasks added yet.</div>")

            # ---- Function to update tasks ----
            def add_task(task, task_list):
                if not task.strip():
                    return task_list, "<div class='task-list'>Enter a task first.</div>", ""

                new_list = (task_list or []) + [task.strip()]

                html = "<div class='task-list'><ul>"
                for t in new_list:
                    html += f"<li>{t}</li>"
                html += "</ul></div>"

                return new_list, html, ""

            add_task_btn.click(
                fn=add_task,
                inputs=[new_task, task_list_state],
                outputs=[task_list_state, task_list_display,new_task]
            )

            # ---- Study plan options ----
            time_input = gr.Slider(1, 8, label="Available Hours")
            difficulty = gr.Dropdown(["Easy", "Medium", "Hard"])
            style = gr.Dropdown(["Pomodoro", "Deep Work", "Short Sessions"])
             # ---- Deadline Input Section ----
            gr.Markdown("<div class='deadline-title'>ADD A DEADLINE</div>")

            deadline_name = gr.Textbox(label="Deadline Name (e.g., Maths Exam)")
            deadline_date = gr.Textbox(label="Deadline Date (YYYY-MM-DD HH:MM)")

            add_deadline_btn = gr.Button("➕ Add Deadline", elem_classes="nav-btn")

            generate_btn = gr.Button("Generate Plan", elem_classes="nav-btn")
            plan_output = gr.Markdown()

            # Function to combine tasks for the LLM (later)
            def prepare_plan(task_list, hours, diff, style, deadline):
                if not task_list:
                    return " Add at least 1 task before generating a plan."

                tasks_joined = "\n".join(task_list)

                try:
                    plan = generate_study_plan(tasks_joined, hours, diff, style)
                except Exception as e:
                    print("LLM ERROR:", e)
                    plan = " AI plan generation failed. Tasks saved anyway."

                save_study_plan(plan, task_list,deadline)
                return plan
            generate_btn.click(
                fn=prepare_plan,
                inputs=[task_list_state, time_input, difficulty, style, deadline_date],
                outputs=plan_output
            ).then(
                fn=load_dashboard,
                inputs=None,
                outputs=[quote_box, kpi_hours, kpi_streak, next_task_box]
            )


        with gr.Tab("Habit Tracker"):
            hours_input = gr.Slider(0, 6, label="Hours Studied")
            tasks_completed = gr.Slider(0, 10, label="Tasks Completed")
            log_btn = gr.Button("Log Session")
            log_output = gr.Markdown()
            log_btn.click(
            fn=log_study_session,
            inputs=[hours_input, tasks_completed],
            outputs=log_output
)

        with gr.Tab("Calender"):
           gr.Markdown("<div class = 'cal-title'>Daily Planner</div>")

        with gr.Tab("Motivation"):

            # ---- Page Title ----
            gr.Markdown("<div class='motivation-title'>STUDY MOTIVATION</div>")

            # ---- Quote Placeholder ----
            motivation_quote = gr.Markdown(
                "<div class='motivation-quote'>Your motivational quote will appear here...</div>"
            )

            refresh_quote_btn = gr.Button("🔄 Refresh Quote", elem_classes="nav-btn")

            gr.Markdown("<hr style='border:1px solid #444; margin-top:20px;'>")

           

            # ---- Deadline List Placeholder ----
            deadlines_list = gr.Markdown(
                "<div class='deadline-list'>Your deadlines will appear here...</div>"
            )

            
    
    theme_toggle.change(
            fn=switch_theme,
            inputs=theme_toggle,
            outputs=css_box,
        )
    
    
    demo.load(
    fn=load_dashboard,
    inputs=None,
    outputs=[quote_box, kpi_hours, kpi_streak, next_task_box]
)



demo.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [2]:
from backend import generate_study_plan

print(
    generate_study_plan(
        tasks="Task A\nTask B",
        hours=2,
        difficulty="Medium",
        style="Pomodoro"
    )
)


# Study Plan for the Day

**Available Study Time:** 2 hours  
**Difficulty Preference:** Medium  
**Study Style Preference:** Pomodoro  

---

## Task Breakdown

- **Task A**: [Time allocated: 50 minutes]
- **Task B**: [Time allocated: 50 minutes]
- **Total Study Time**: 1 hour 40 minutes
- **Break Time**: 20 minutes

---

## Study Schedule

### 1. Session 1
**Time:** 00:00 - 00:25  
**Task:** Task A  
**Focus:** Dive into the material or project. Aim to cover the core concepts or sections you find moderately challenging.  

### Break 1
**Time:** 00:25 - 00:30  
**Duration:** 5 minutes  
**Activity:** Step away from your study area, stretch, grab a drink, or take a few deep breaths.  

---

### 2. Session 2
**Time:** 00:30 - 00:55  
**Task:** Task A (continued)  
**Focus:** Continue working on Task A, solidifying your understanding and completing the task.  

### Break 2
**Time:** 00:55 - 01:00  
**Duration:** 5 minutes  
**Activity:** Relax, walk around, or clear your mind to prepare 

Saving plan to: c:\Users\emman\Documents\App\Developing-apps-with-AI-project\app\current_plan.json


In [5]:
from backend import save_study_plan

save_study_plan("TEST PLAN", ["Task A", "Task B"])


Saving plan to: c:\Users\emman\Documents\App\Developing-apps-with-AI-project\app\current_plan.json


In [5]:
import os
print("CWD:", os.getcwd())
import backend
print(backend.__file__)

print("FILES:", os.listdir())


CWD: c:\Users\emman\Documents\App\Developing-apps-with-AI-project\app
c:\Users\emman\Documents\App\Developing-apps-with-AI-project\app\backend.py
FILES: ['app.py', 'app_practice.ipynb', 'assets', 'backend.py', 'current_plan.json', 'prompts.py', '__pycache__']
